# Task 2 function

In [1]:
def floyd(graph, no_path=-1):
    '''
    Floyd–Warshall algorithm.
    
    The graph defines by a matrix with 
    dimensions NxN. Element graph[i][j] 
    is equal to the length of the road 
    between city i and city j, 
    if it exists, and -1 otherwise (by default).
    
    Parameters
    ----------
    %(input)s
    graph : array-like
        The graph matrix NxN.
    no_path : int, optional
        The no path value between cities.
    
    Returns
    -------
    graph : ndarray
        The shortest path matrix.
    '''
    
    from numpy import array
    from math import inf
    
    graph = array(graph).astype(float)
    graph[graph == no_path] = inf
    
    n = len(graph)
    for k in range(0,n):
        for i in range(0,n):
            for j in range(0,n):
                if graph[i,k] < inf and graph[k,j] < inf and graph[i,j] > graph[i,k] + graph[k,j]:
                    graph[i,j] = graph[i,k] + graph[k,j]
                    
    return graph

# Task 2 examples

As an example we consider the path matrix
$
A = \begin{pmatrix}
0 & 100 & -1 & 1\\
100 & 0 & 50 & 3\\
-1 & 50 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$.

In [2]:
A = [[0, 100, -1, 1],[100, 0, 50, 3],[-1, 50, 0, 20],[1, 3, 20, 0]]
A

[[0, 100, -1, 1], [100, 0, 50, 3], [-1, 50, 0, 20], [1, 3, 20, 0]]

Apply Floyd's algorithm to the $A$:

In [3]:
floyd(A)

array([[ 0.,  4., 21.,  1.],
       [ 4.,  0., 23.,  3.],
       [21., 23.,  0., 20.],
       [ 1.,  3., 20.,  0.]])

And we get the result:
$$
A_{min} = \begin{pmatrix}
0 & 4 & 21 & 1\\
4 & 0 & 23 & 3\\
21 & 23 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$$